In [52]:
import pandas as pd
import numpy as np

In [53]:
!conda install -c conda-forge pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

from geopy.geocoders import Nominatim 

import requests 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: \ 
  - anaconda::ca-certificates-2018.03.07-0, anaconda::openssl-1.0.2o-h26aff7b_0
  - anaconda::openssl-1.0.2o-h26aff7b_0, defaults::ca-certificates-2018.03.07-0
  - anaconda::ca-certificates-2018.03.07-0, defaults::openssl-1.0.2o-h26aff7b_0
  - defaults::ca-certificates-2018.03.07-0, defaults::openssl-1.0.2o-h26aff7bdone

# All requested packages already installed.

Solving environment: \ 
  - anaconda::ca-certificates-2018.03.07-0, anaconda::openssl-1.0.2o-h26aff7b_0
  - anaconda::openssl-1.0.2o-h26aff7b_0, defaults::ca-certificates-2018.03.07-0
  - anaconda::ca-certificates-2018.03.07-0, defaults::openssl-1.0.2o-h26aff7b_0
  - defaults::ca-certificates-2018.03.07-0, defaults::openssl-1.0.2o-h26aff7bdone

# All requested packages already installed.



In [239]:
df = pd.read_excel('Canlat.xls')

In [240]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]))

The dataframe has 4 boroughs and 72 neighborhoods.


In [241]:
df['Neighborhood']=df['Neighborhood'].str.replace(')','')
df['Neighborhood']=df['Neighborhood'].str.replace('/',',')

In [242]:
address = 'Toronto, ON'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))


/anaconda3/lib/python3.6/site-packages/geopy/geocoders/osm.py:138: UserWarning: Using Nominatim with the default "geopy/1.16.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [243]:
# Get data only for Toronto
toronto = df[df['Borough'].str.contains('Toronto', regex=False)].reset_index(drop=True)
toronto.shape

(72, 5)

In [244]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, label in zip(toronto['lat'], toronto['lng'], toronto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='Brown',
        fill=True,
        fill_opacity=0.8,
        ).add_to(map_toronto)
map_toronto

In [245]:
CLIENT_ID = 'My_Client_ID'
CLIENT_SECRET = 'My_Client_Secret'
VERSION = '21082018'

In [246]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # Request foursquare with url
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [247]:
LIMIT=100
toronto_venues = getNearbyVenues(names=toronto['Neighborhood'],
                                   latitudes=toronto['lat'],
                                   longitudes=toronto['lng']
                                  )

Harbourfront
Regent Park
Ryerson
Garden District
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
The Danforth West
Riverdale
Design Exchange
Toronto Dominion Centre
Brockton
Exhibition Place
Parkdale Village
The Beaches West
India Bazaar
Commerce Court
Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North
Forest Hill West
High Park
The Junction South
North Toronto West
The Annex
North Midtown
Yorkville
Parkdale
Roncesvalles
Davisville
Harbord
University of Toronto
Runnymede
Swansea
Moore Park
Summerhill East
Chinatown
Grange Park
Kensington Market
Deer Park
Forest Hill SE
Rathnelly
South Hill
Summerhill West
CN Tower
Bathurst Quay
Island airport
Harbourfront West
King and Spadina
Railway Lands
South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown
First Canadian Place
Underground city
Church and Wellesley
Business 

In [248]:

toronto_venues = toronto_venues[toronto_venues['Venue Category'] != 'Neighborhood']

toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.insert (0,'Neighborhood',toronto_venues['Neighborhood'])
toronto_onehot.head()

,Neighborhood,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Cafeteria,Café,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pilates Studio,Pizza Place,Playground,Plaza,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rest Area,Restaurant,River,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Transportation Service,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [249]:
toronto_onehot.shape

(4836, 232)

In [250]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.shape

(72, 232)

In [251]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
                 venue  freq
0          Coffee Shop  0.07
1           Steakhouse  0.04
2  American Restaurant  0.04
3                 Café  0.04
4      Thai Restaurant  0.04


----Bathurst Quay----
                venue  freq
0          Restaurant  0.08
1         Coffee Shop  0.06
2  Italian Restaurant  0.04
3               Hotel  0.04
4                 Gym  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2  Seafood Restaurant  0.04
3                Café  0.04
4         Cheese Shop  0.04


----Brockton----
                           venue  freq
0                    Coffee Shop  0.09
1         Furniture / Home Store  0.07
2                     Restaurant  0.07
3                      Gift Shop  0.04
4  Vegetarian / Vegan Restaurant  0.04


----Business reply mail Processing Centre969 Eastern----
           venue  freq
0  Burrito Place  0.08
1   Burger Joint  0.08
2    Supermarket  0.08
3     Steakhouse  0.04


                venue  freq
0    Greek Restaurant  0.25
1         Coffee Shop  0.09
2      Ice Cream Shop  0.07
3  Italian Restaurant  0.05
4           Bookstore  0.05


----Roncesvalles----
                 venue  freq
0       Clothing Store  0.06
1  American Restaurant  0.05
2          Coffee Shop  0.04
3           Steakhouse  0.03
4                 Café  0.03


----Rosedale----
             venue  freq
0             Park  0.50
1       Playground  0.25
2            Trail  0.25
3  Other Nightlife  0.00
4    Movie Theater  0.00


----Roselawn----
             venue  freq
0     Home Service   0.5
1           Garden   0.5
2   Adult Boutique   0.0
3  Other Nightlife   0.0
4    Movie Theater   0.0


----Runnymede----
              venue  freq
0       Coffee Shop  0.11
1              Café  0.08
2  Sushi Restaurant  0.05
3       Pizza Place  0.05
4      Dessert Shop  0.03


----Ryerson----
                 venue  freq
0          Coffee Shop  0.08
1       Clothing Store  0.07
2               

In [252]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [253]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Steakhouse,American Restaurant,Café,Thai Restaurant,Restaurant,Cosmetics Shop,Hotel,Bar,Bakery
1,Bathurst Quay,Restaurant,Coffee Shop,Italian Restaurant,Gym,Hotel,Sandwich Place,Vegetarian / Vegan Restaurant,French Restaurant,Yoga Studio,Theater
2,Berczy Park,Coffee Shop,Cocktail Bar,Steakhouse,Restaurant,Beer Bar,Café,Seafood Restaurant,Farmers Market,Cheese Shop,Bakery
3,Brockton,Coffee Shop,Restaurant,Furniture / Home Store,Vegetarian / Vegan Restaurant,Bar,Hotel,Gift Shop,Sandwich Place,Italian Restaurant,Gym
4,Business reply mail Processing Centre969 Eastern,Supermarket,Burrito Place,Burger Joint,Fast Food Restaurant,Laser Tag,Electronics Store,Fried Chicken Joint,Coffee Shop,Cafeteria,Bank


In [254]:
# K mean clustering for Toronto neighborhoods

In [255]:
kclusters=5
toronto_grouped_clustering = toronto_grouped.drop(columns='Neighborhood')
kmeans_tot = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
kmeans_tot.cluster_centers_

array([[-2.71050543e-20, -2.71050543e-20,  9.93048170e-03, ...,
         9.52380952e-04, -2.16840434e-19,  6.38554656e-03],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 2.71002710e-04,  2.71002710e-04,  2.05838155e-02, ...,
         0.00000000e+00,  2.02020202e-03,  5.47981036e-03],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [256]:
toronto.shape

(72, 5)

In [259]:

toronto_merged = toronto
toronto_merged['Cluster Labels'] = kmeans_tot.labels_
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.head()

,Postcode,Borough,Neighborhood,lat,lng,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,3,Coffee Shop,Park,Café,Bakery,Theater,Gym / Fitness Center,Restaurant,Mexican Restaurant,Pub,Breakfast Spot
1,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636,3,Coffee Shop,Park,Café,Bakery,Theater,Gym / Fitness Center,Restaurant,Mexican Restaurant,Pub,Breakfast Spot
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937,3,Coffee Shop,Clothing Store,Café,Italian Restaurant,Japanese Restaurant,Bar,Cosmetics Shop,Theater,Diner,Bubble Tea Shop
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937,3,Coffee Shop,Clothing Store,Café,Italian Restaurant,Japanese Restaurant,Bar,Cosmetics Shop,Theater,Diner,Bubble Tea Shop
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,3,Pub,Coffee Shop,Yoga Studio,Electronics Store,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant


In [260]:
# create map 
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['lat'], toronto_merged['lng'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [261]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Bar,Japanese Restaurant,Burger Joint,Chinese Restaurant,Thai Restaurant,Bubble Tea Shop,Spa
9,King,Coffee Shop,Steakhouse,American Restaurant,Café,Thai Restaurant,Restaurant,Cosmetics Shop,Hotel,Bar,Bakery
12,Dufferin,Coffee Shop,Café,Pizza Place,Sushi Restaurant,Smoothie Shop,Sandwich Place,Pub,Food,Burrito Place,Gastropub
16,Little Portugal,Clothing Store,Bar,Coffee Shop,Grocery Store,Discount Store,Italian Restaurant,Café,Electronics Store,Arts & Crafts Store,Furniture / Home Store
17,Trinity,Clothing Store,Bar,Coffee Shop,Grocery Store,Discount Store,Italian Restaurant,Café,Electronics Store,Arts & Crafts Store,Furniture / Home Store
25,The Beaches West,Sandwich Place,Pub,Coffee Shop,Steakhouse,Movie Theater,Fast Food Restaurant,Brewery,Italian Restaurant,Sushi Restaurant,Liquor Store
26,India Bazaar,Sandwich Place,Pub,Coffee Shop,Steakhouse,Movie Theater,Fast Food Restaurant,Brewery,Italian Restaurant,Sushi Restaurant,Liquor Store
29,Studio District,Café,Coffee Shop,Gastropub,American Restaurant,Bakery,Italian Restaurant,Yoga Studio,Brewery,Latin American Restaurant,Bookstore
30,Lawrence Park,Bus Line,Dim Sum Restaurant,Swim School,Park,Yoga Studio,Ethiopian Restaurant,Food,Flower Shop,Flea Market,Fish Market
37,North Toronto West,Clothing Store,American Restaurant,Coffee Shop,Tea Room,Burger Joint,Café,Breakfast Spot,Steakhouse,Seafood Restaurant,Chinese Restaurant


In [262]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Design Exchange,Coffee Shop,Hotel,Café,Deli / Bodega,Italian Restaurant,Bakery,Restaurant,Gastropub,American Restaurant,Concert Hall
22,Brockton,Coffee Shop,Restaurant,Furniture / Home Store,Vegetarian / Vegan Restaurant,Bar,Hotel,Gift Shop,Sandwich Place,Italian Restaurant,Gym
35,High Park,Italian Restaurant,Bar,Coffee Shop,Café,Bakery,Antique Shop,Nail Salon,Grocery Store,Thai Restaurant,Yoga Studio
48,Moore Park,Coffee Shop,Italian Restaurant,Café,Sushi Restaurant,Thai Restaurant,Pub,Gym,Pizza Place,Bagel Shop,Sandwich Place


In [263]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
49,Summerhill East,Coffee Shop,Italian Restaurant,Café,Sushi Restaurant,Thai Restaurant,Pub,Gym,Pizza Place,Bagel Shop,Sandwich Place


In [264]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Harbourfront,Coffee Shop,Park,Café,Bakery,Theater,Gym / Fitness Center,Restaurant,Mexican Restaurant,Pub,Breakfast Spot
1,Regent Park,Coffee Shop,Park,Café,Bakery,Theater,Gym / Fitness Center,Restaurant,Mexican Restaurant,Pub,Breakfast Spot
2,Ryerson,Coffee Shop,Clothing Store,Café,Italian Restaurant,Japanese Restaurant,Bar,Cosmetics Shop,Theater,Diner,Bubble Tea Shop
3,Garden District,Coffee Shop,Clothing Store,Café,Italian Restaurant,Japanese Restaurant,Bar,Cosmetics Shop,Theater,Diner,Bubble Tea Shop
4,The Beaches,Pub,Coffee Shop,Yoga Studio,Electronics Store,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant
5,Berczy Park,Coffee Shop,Cocktail Bar,Steakhouse,Restaurant,Beer Bar,Café,Seafood Restaurant,Farmers Market,Cheese Shop,Bakery
7,Christie,Grocery Store,Café,Park,Nightclub,Coffee Shop,Convenience Store,Diner,Restaurant,Italian Restaurant,Athletics & Sports
8,Adelaide,Coffee Shop,Steakhouse,American Restaurant,Café,Thai Restaurant,Restaurant,Cosmetics Shop,Hotel,Bar,Bakery
10,Richmond,Coffee Shop,Steakhouse,American Restaurant,Café,Thai Restaurant,Restaurant,Cosmetics Shop,Hotel,Bar,Bakery
11,Dovercourt Village,Coffee Shop,Café,Pizza Place,Sushi Restaurant,Smoothie Shop,Sandwich Place,Pub,Food,Burrito Place,Gastropub


In [265]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
60,Island airport,Restaurant,Coffee Shop,Italian Restaurant,Gym,Hotel,Sandwich Place,Vegetarian / Vegan Restaurant,French Restaurant,Yoga Studio,Theater
